## DIR structure
* ./ensemble(Avg).py
* ./exec_KFold.py
* ./inference.py
* ./lightning.py
* ./stratifiedKfold_pl_data.py
* ./runner.py
* ./model/models.py
* ./lightning_logs/*
* ./prediction/*.csv
* ./data/DATA


### ensemble(Avg).py

In [ ]:
# ./ensemble(Avg).py
import glob
import pandas as pd

from easydict import EasyDict
from sklearn import preprocessing
from tqdm import tqdm

from data_loader import *
from lightning import LightningRunner
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from model.models import BaseModel
import numpy as np


args = EasyDict()

args.img_size = 544
args.batch_size = 4


test_transform = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

le = preprocessing.LabelEncoder()

all_img_list = glob.glob('./data/train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])
df['label'] = le.fit_transform(df['label'])

test = pd.read_csv('/root/Competitions/DACON/Papering_clf/data/test.csv')
path = test['img_path'].values
path = [f'/root/Competitions/DACON/Papering_clf/data/test/{file.split("/")[-1]}' for file in path]


test_dataset = CustomDataset(path, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

CV_path = '/root/Competitions/DACON/Papering_clf/lightning_logs/2023-05-20'
ckpts = glob.glob(f'{CV_path}/*/checkpoints/*')

inferences = []
DEVICE = 'cuda:0'

for ckpt in ckpts:
    model = BaseModel(19).to(DEVICE)
    pl_runner = LightningRunner.load_from_checkpoint(ckpt, network=model, args=args).to(DEVICE)
    # pl_runner
    pl_runner.eval()

    inference  = []
    for x in tqdm(test_loader):
        x = x.to(DEVICE)

        pred = pl_runner.model(x)
        inference += pred.detach().cpu().numpy().tolist()

        del x 
    
    inferences.append(inference)

inferences = np.mean(np.array(inferences), axis=0)
inferences = np.argmax(inferences, axis=1)

inferences = le.inverse_transform(inferences)
submit = pd.read_csv('/root/Competitions/DACON/Papering_clf/data/sample_submission.csv')
submit['label'] = inferences
submit.to_csv('/root/Competitions/DACON/Papering_clf/prediction/5fold_convNeXt-Large_aug-realP-p544.csv', index=False)

### exec_KFold.py

In [ ]:
# exec_KFold.py 
from datetime import datetime, timezone, timedelta

if __name__ == '__main__':

    from easydict import EasyDict
    from lightning_fabric.utilities.seed import seed_everything
    from pytorch_lightning import Trainer
    from pytorch_lightning.strategies.ddp import DDPStrategy
    from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, LearningRateFinder
    from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

    import albumentations as A
    from albumentations.pytorch.transforms import ToTensorV2

    from lightning import LightningRunner
    from data_loader import *
    from model.models import *
    from stratifiedKfold_pl_data import KFold_pl_DataModule



    args = EasyDict()

    args.img_size = 512
    args.val_img_size = 544 

    args.batch_size = 16
    args.epochs = 80
    args.init_lr = 4e-5
    args.weight_decay = 0.05
    args.seed = 41
    

    seed_everything(args.seed)
    
    
    train_transform_4_origin = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.AdvancedBlur(),
                            A.ColorJitter(),
                            A.GaussNoise(),
                            A.OpticalDistortion(distort_limit=(-0.3, 0.3), shift_limit=0.5, p=0.5),
                            A.HorizontalFlip(),
                            A.Affine(scale=(0.9, 2), translate_percent=(-0.1, 0.1), rotate=(-10, 10), shear=(-20,20)),
                            A.ElasticTransform(alpha=300),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
    
    test_transform = A.Compose([
                            A.Resize(args.val_img_size, args.val_img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
    
    num_split = 5 
    KST = timezone(timedelta(hours=9))
    start = datetime.now(KST)
    _day = str(start)[:10]
    for k_idx in range(num_split):
        pl_dataFolder = KFold_pl_DataModule(data_dir='./proc_data/train/*/*',
                            k_idx=k_idx,
                            num_split=num_split,
                            split_seed=args.seed,
                            batch_size=args.batch_size,
                            num_workers=4,
                            pin_memory=False,
                            persistent_workers=True,
                            train_transform=train_transform_4_origin,
                            val_transform=test_transform)

        pl_dataFolder.setup(stage=None)
        model = BaseModel(pl_dataFolder.num_cls)
        pl_runner = LightningRunner(model, args)
        lr_monitor = LearningRateMonitor(logging_interval='step')
        
        checkpoint_callback = ModelCheckpoint(
            monitor='avg_f1',
            filename=f'{model.__class__.__name__}'+'-{epoch:03d}-{train_loss:.4f}-{avg_f1:.4f}',
            mode='max'
        )

        logger = TensorBoardLogger(
            save_dir='.',
            # version='LEARNING CHECK',
            version=f'{_day}/[{k_idx+1} Fold] -m convnext_large, -d realP, -t GV -opt AdamP || lr=[{args.init_lr}] img=[{args.img_size}] bz=[{args.batch_size}] 2gpu'
            )

        trainer = Trainer(
            max_epochs=args.epochs,
            devices=[2,3],
            accelerator='gpu',
            precision='16-mixed',
            # strategy=DDPStrategy(find_unused_parameters=False),
            callbacks=[lr_monitor, checkpoint_callback],
            # check_val_every_n_epoch=2,
            check_val_every_n_epoch=2,
            # log_every_n_steps=1,
            logger=logger,
            # auto_lr_find=True
            # accumulate_grad_batches=2
            )

        trainer.fit(
            model= pl_runner,
            # train_dataloaders=train_loader,
            # val_dataloaders=val_loader
            datamodule=pl_dataFolder
        )

    # fold iteration END
    print(f'execution done --- time cost: [{datetime.now(KST) - start}]')


### inference.py

In [ ]:
# ./inference.py
import glob
import pandas as pd

from easydict import EasyDict
from sklearn import preprocessing
from tqdm import tqdm

from data_loader import *
from lightning import LightningRunner
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from model.models import BaseModel


args = EasyDict()

args.img_size = 512
args.batch_size = 1


test_transform = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

le = preprocessing.LabelEncoder()

all_img_list = glob.glob('./data/train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])
df['label'] = le.fit_transform(df['label'])

test = pd.read_csv('/root/Competitions/DACON/Papering_clf/data/test.csv')
path = test['img_path'].values
path = [f'/root/Competitions/DACON/Papering_clf/data/test/{file.split("/")[-1]}' for file in path]


test_dataset = CustomDataset(path, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

model = BaseModel(19)

ckpt = '/root/Competitions/DACON/Papering_clf/lightning_logs/[1 Fold] -m ConvNeXt_base, -d A, -t GV || lr=[8e-05], img=[512], bz=[32]/checkpoints/BaseModel-epoch=059-train_loss=0.7751-avg_f1=0.8802.ckpt'
pl_runner = LightningRunner.load_from_checkpoint(ckpt, network=model, args=args)

DEVICE = 'cuda:1'
pl_runner.to(DEVICE)
pl_runner.eval()

inferences = []
for x in tqdm(test_loader):
    x = x.to(DEVICE)

    pred = pl_runner.model(x)
    inferences += pred.argmax(1).detach().cpu().numpy().tolist()

    del x 

inferences = le.inverse_transform(inferences)
submit = pd.read_csv('/root/Competitions/DACON/Papering_clf/data/sample_submission.csv')
submit['label'] = inferences
submit.to_csv('/root/Competitions/DACON/Papering_clf/prediction/[1f] -m ConvNeXt_b, -d A, -t GV || lr=[8e-05], img=[512], bz=[32], pim=[512](-epc=059-loss=0.7751-f1=0.8802).csv', index=False)

### lightning.py

In [ ]:
# ./lightning.py
import torch
import torch.nn as nn
import torch.optim as optim

import pytorch_lightning as pl
from timm.loss import AsymmetricLossSingleLabel
from timm.data import Mixup
from timm.data.random_erasing import RandomErasing

from sklearn.metrics import classification_report, f1_score
import numpy as np
from functools import partial
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
from model.apollo import Apollo
from adamp import AdamP

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
    
        ce_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-ce_loss)  # CE(pt) = -log(pt) --> -ce_loss = log(pt) --> exp(log(pt)) --> pt
        F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss


import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

class LightningRunner(pl.LightningModule):
    def __init__(self, network, args) -> None:
        super().__init__()

        self.model = network
        self.loss = FocalLoss()
        # self.loss = LADELoss(args.num_cls, args.prior)
        self.args = args
        self.traget_names=[f'cls{idx}' for idx in range(19)]

        mixup_args = {
            'mixup_alpha': 0.3,
            'cutmix_alpha': 0.5,
            'cutmix_minmax': None,
            'prob': 1.0,
            'switch_prob': 0.5,
            'mode': 'batch',
            'label_smoothing': 0.1,
            'num_classes': 19}

        self.mixup_fn = Mixup(**mixup_args)
        # self.rand_erase = RandomErasing(probability=0.5)

        # val archieve
        self.preds = []
        self.true_labels = []
        self.loss_log = []


    def configure_optimizers(self):
        optimizer = AdamP(params=self.parameters(), lr=self.args.init_lr, betas=[0.9, 0.999], weight_decay=0.05)
        lr_scheduler = CosineAnnealingWarmupRestarts(optimizer=optimizer, first_cycle_steps=self.args.epochs, max_lr=self.args.init_lr, min_lr=self.args.init_lr*0.001, warmup_steps=self.args.epochs//10, gamma=0.8)
        return [optimizer], [lr_scheduler]
    
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        # x = self.rand_erase(x)

        if (x.shape[0] % 2 == 0):
                x, y = self.mixup_fn(x, y)

        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True, batch_size=self.args.batch_size, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        self._shared_eval_step(batch, batch_idx)

    def on_validation_epoch_end(self) -> None:

        val_loss = np.mean(self.loss_log)
        
        avg_f1 = f1_score(self.true_labels, self.preds, average='weighted')
        self.log_dict({'val_loss':val_loss, 'avg_f1':avg_f1}, prog_bar=True, sync_dist=True)

        if len(np.unique(self.true_labels)) == 19:
            report = classification_report(self.true_labels, self.preds, target_names=self.traget_names)
            report_lines = report.split('\n')

            for line in report_lines[2: 2+len(self.traget_names)]:
                cls_name, *cls_metrics, support = line.split()
                self.log_dict({
                    f'precision/{cls_name}': torch.Tensor([float(cls_metrics[0])]),
                    f'recall/{cls_name}': torch.Tensor([float(cls_metrics[1])]),
                    f'f1-score/{cls_name}': torch.Tensor([float(cls_metrics[2])]),
                    f'support/{cls_name}': torch.Tensor([float(support)]),
                })
            

        self.loss_log.clear()
        self.true_labels.clear()
        self.preds.clear()

    def _shared_eval_step(self, batch, batch_idx):
        imgs,  labels = batch

        y_hat = self.model(imgs)
        loss = self.loss(y_hat, labels)

        self.preds += y_hat.argmax(1).detach().cpu().numpy().tolist()
        self.true_labels += labels.detach().cpu().numpy().tolist()
        self.loss_log.append(loss.item())


### data_loader.py

In [ ]:
# ./data_loader.py
import cv2

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

### stratifiedKfold_pl_data.py

In [ ]:
# ./stratifiedKfold_pl_data.py 
import os
import pytorch_lightning as pl

import glob
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing

from data_loader import CustomDataset

class KFold_pl_DataModule(pl.LightningDataModule):
    def __init__(self,
                 data_dir: str = None,
                 k_idx: int =1, # fold index
                 num_split: int = 5, # fold number, if k=1 then return the whole data
                 split_seed: int = 41,
                 batch_size: int = 2, 
                 num_workers: int = 0,
                 pin_memory: bool = False,
                 persistent_workers: bool=True,
                 train_transform=None,
                 val_transform =None
                 ) -> None:
        super().__init__()
        self.save_hyperparameters(logger=False)

        self.train_data = None
        self.val_data = None
        self.num_cls = 0

    def setup(self, stage: str) -> None:
        if not self.train_data and not self.val_data:
            all_img_list = glob.glob(self.hparams.data_dir)
            df = pd.DataFrame(columns=['img_path', 'label'])
            df['img_path'] = all_img_list
            df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

            kf = StratifiedKFold(n_splits=self.hparams.num_split,
                       shuffle=True,
                       random_state=self.hparams.split_seed)
            
            all_splits = [k for k in kf.split(df, df['label'])]
            train_idx, val_idx = all_splits[self.hparams.k_idx]
            train_idx, val_idx = train_idx.tolist(), val_idx.tolist()

            train = df.iloc[train_idx]
            val = df.iloc[val_idx]

            le = preprocessing.LabelEncoder()
            le.fit(df['label'])
            train['label'] = le.transform(train['label'])
            val['label'] = le.transform(val['label'])
            self.num_cls = len(le.classes_)


            
            self.train_data = CustomDataset(train['img_path'].values, train['label'].values, self.hparams.train_transform)
            self.val_data = CustomDataset(val['img_path'].values, val['label'].values, self.hparams.val_transform)

    def train_dataloader(self):
        return DataLoader(self.train_data,
                          batch_size=self.hparams.batch_size,
                          shuffle=True,
                          num_workers=self.hparams.num_workers,
                          persistent_workers=self.hparams.persistent_workers,
                          pin_memory=self.hparams.pin_memory)
    
    def val_dataloader(self):
        return DataLoader(self.val_data,
                          batch_size=self.hparams.batch_size//2,
                          shuffle=False,
                          num_workers=self.hparams.num_workers,
                          persistent_workers=self.hparams.persistent_workers,
                          pin_memory=self.hparams.pin_memory)


### runner.py

In [ ]:
#  ./runner.py
if __name__ == '__main__':

    from easydict import EasyDict

    from sklearn import preprocessing
    from sklearn.model_selection import train_test_split
    from lightning_fabric.utilities.seed import seed_everything

    from pytorch_lightning import Trainer
    from pytorch_lightning.strategies.ddp import DDPStrategy
    from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, LearningRateFinder
    from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
    from pytorch_lightning import tuner as Tuner

    import albumentations as A
    from albumentations.pytorch.transforms import ToTensorV2

    import glob    
    import pandas as pd


    from lightning import LightningRunner
    from data_loader import *
    from model.models import *
    from torch.utils.data import DataLoader

    args = EasyDict()

    args.img_size = 368

    args.batch_size = 32
    args.epochs = 80
    args.init_lr = 8e-5
    args.weight_decay = 0.05

    args.seed = 1120
    

    seed_everything(args.seed)

    

    all_img_list = glob.glob('./aug_data/train/*/*')
    df = pd.DataFrame(columns=['img_path', 'label'])
    df['img_path'] = all_img_list
    df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])
 

    train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=args.seed)

    le = preprocessing.LabelEncoder()
    train['label'] = le.fit_transform(train['label'])
    val['label'] = le.transform(val['label'])

    args.prior = df.label.value_counts().to_frame().sort_index().values
    args.num_cls = len(le.classes_)


    train_transform_4_origin = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.AdvancedBlur(),
                            A.ColorJitter(),
                            A.GaussNoise(),
                            A.OpticalDistortion(distort_limit=(-0.3, 0.3), shift_limit=0.5, p=0.5),
                            A.HorizontalFlip(),
                            A.Affine(scale=(0.9, 2), translate_percent=(-0.1, 0.1), rotate=(-10, 10), shear=(-20,20)),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.ElasticTransform(alpha=300),
                            ToTensorV2()
                            ])
    
    test_transform = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])


    train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform_4_origin)
    train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=4)

    val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size//2, shuffle=False, num_workers=4)

    model = BaseModel(len(le.classes_))
    pl_runner = LightningRunner(model, args)
    lr_monitor = LearningRateMonitor(logging_interval='step')

    
    # lr_finder = LearningRateFinder(num_training_steps=8000)
    checkpoint_callback = ModelCheckpoint(
        monitor='avg_f1',
        filename=f'{model.__class__.__name__}'+'-{epoch:03d}-{train_loss:.4f}-{avg_f1:.4f}',
        mode='max'
    )

    logger = TensorBoardLogger(
        save_dir='.',
        version='LEARNING CHECK',
        # version=f'[S.7] --m ConvNeXt, --d A, --t GV --L F long || lr=[{args.init_lr}], img=[{args.img_size}], bz=[{args.batch_size}]'
        )

    trainer = Trainer(
        max_epochs=args.epochs,
        devices=[0],
        accelerator='gpu',
        precision='16-mixed',
        # strategy=DDPStrategy(find_unused_parameters=False),
        callbacks=[lr_monitor, checkpoint_callback],
        # check_val_every_n_epoch=2,py
        check_val_every_n_epoch=2,
        # log_every_n_steps=1,
        logger=logger,
        # auto_lr_find=True
        # accumulate_grad_batches=2
        )


    trainer.fit(
        model= pl_runner,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
        # datamodule=pl_data
    )


### models.py

In [ ]:
# ./model/models.py
import torch.nn as nn
import torchvision.models as models

class BaseModel(nn.Module):
    def __init__(self, num_classes:int):
        super(BaseModel, self).__init__()
        
        self.backbone = models.convnext_large(weights=models.ConvNeXt_Large_Weights.DEFAULT)
        self.norm = nn.LayerNorm(1000)
        self.act = nn.SiLU()
        self.drop = nn.Dropout1d()
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.classifier(x)
        return x
    
    
# def replace_module(modules:nn.Module, target, source):
#         for name, child in modules.named_children():
#             if isinstance(child, target):
#                 modules._modules[name] = source()
#             # elif isinstance(child, nn.Sequential):
#             else: 
#                 replace_module(child, target, source)
